In [ ]:
import numpy as np

from RushHour4.setup import Map, Environment

In [2]:
map = Map(6, 9)
map.index
map.coordinate
map.view()

0,1,2,3,4,5,6,7,8
9,10,11,12,13,14,15,16,17
18,19,20,21,22,23,24,25,26
27,28,29,30,31,32,33,34,35
36,37,38,39,40,41,42,43,44
45,46,47,48,49,50,51,52,53


o,o,o,o,o,o,o,x,x
o,o,o,x,x,o,o,o,o
o,o,x,x,o,x,o,o,o
o,o,x,x,o,o,x,x,o
o,o,o,o,o,o,x,x,o
o,o,o,o,o,o,o,o,o


In [15]:
env = Environment(map)

env.valid_states(22, index=True)

['down']

In [7]:
env.next('right', 45, index=True)

46